# Test csv file import   


In [1]:
import pandas as pd
import numpy as np

Pour adapter la routine d'importation quand on voit des problèmes

In [45]:
ref_file_path='media/House1_2025.csv'
example_file_path='media/House1_2025_kWh.csv'
#example_file_path='media/example_smartmeter (3).csv'
used_file = ref_file_path
used_file = example_file_path

In [46]:
try:
    df_raw = pd.read_csv(used_file,  
                        )
    #TODO: ici faire des cas spécifiques pour les formats des GRDs

except Exception as e:
    raise ValueError(f"Impossible de lire le CSV : {e}")


print(df_raw.head())
df_raw.info()

                  Time Consumption kWh Consumption kW
0  2025-01-01 00:15:00        0.207522       0.830088
1  2025-01-01 00:30:00    0.1700726667   0.6802906667
2  2025-01-01 00:45:00    0.1525166667   0.6100666667
3  2025-01-01 01:00:00    0.1810751667   0.7243006667
4  2025-01-01 01:15:00    0.1737063333   0.6948253333
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35040 entries, 0 to 35039
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Time             35040 non-null  object
 1   Consumption kWh  35040 non-null  object
 2   Consumption kW   35040 non-null  object
dtypes: object(3)
memory usage: 821.4+ KB


In [47]:
columns =df_raw.columns
datetime_col = columns[0]
dt = pd.to_datetime(df_raw[datetime_col], errors="raise")
df_raw[datetime_col] = dt
df = df_raw.set_index(datetime_col).sort_index()
df.head()

,Consumption kWh,Consumption kW
Time,,
2025-01-01 00:15:00,0.207522,0.830088
2025-01-01 00:30:00,0.1700726667,0.6802906667
2025-01-01 00:45:00,0.1525166667,0.6100666667
2025-01-01 01:00:00,0.1810751667,0.7243006667
2025-01-01 01:15:00,0.1737063333,0.6948253333


In [40]:
print(df.dtypes)

# Détection d'une colonne de consommation (numérique)
numeric_cols = df.select_dtypes(include="number").columns
print("colonne numérique détectée:", numeric_cols)

if len(numeric_cols) == 0:
    #essayons de convertir la première colonne non numérique en numérique:

    non_numeric_cols = df.select_dtypes(include="object").columns
    print(non_numeric_cols)
    if len(non_numeric_cols) > 0:
        col_to_convert = non_numeric_cols[0]
        df[col_to_convert] = pd.to_numeric(df[col_to_convert], errors="coerce")
        numeric_cols = df.select_dtypes(include="number").columns
        print("Après conversion, colonnes numériques détectées:", numeric_cols)
        print("Quantité d'erreur de conversion de la colonne:", df[col_to_convert].isna().sum())
        #Si il y moins de 1% de NaN, on peut considérer que la conversion a réussi et on remplace les NaN par 0
        threshold = 0.01 * len(df)
        if df[col_to_convert].isna().sum() <= threshold:
            df[col_to_convert].fillna(0, inplace=True)
        else:
            raise ValueError(f"Conversion de la colonne '{col_to_convert}' échouée : trop de NaN ({df[col_to_convert].isna().sum()}) \nMerci de fournir un fichier avec au moins une colonne de valeurs numériques.")
    #         raise ValueError(
    #             "Aucune colonne numérique trouvée pour la consommation.\n"
    #             "Merci de fournir un fichier avec au moins une colonne de valeurs numériques."
    #         )


Consumption kWh    object
Consumption kW     object
dtype: object
colonne numérique détectée: Index([], dtype='object')
Index(['Consumption kWh', 'Consumption kW'], dtype='object')
Après conversion, colonnes numériques détectées: Index(['Consumption kWh'], dtype='object')
Quantité d'erreur de conversion de la colonne: 15


C:\Users\moix_\AppData\Local\Temp\ipykernel_29160\1758089890.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col_to_convert].fillna(0, inplace=True)


In [48]:
col_to_convert = non_numeric_cols[0]
col_to_convert
print(df[col_to_convert].isna().sum())

0
